In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
from glob import glob
from tqdm import tqdm
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from shutil import copy
from sklearn.model_selection import StratifiedKFold
import torch
from tempfile import TemporaryDirectory

from albk.data.utils import idx_to_locate
use_disjoint_files = False


import torch
import torch.nn as nn

from glob import glob
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
from tqdm import tqdm
import pandas as pd
from joblib import Parallel, delayed
from itertools import product
import torch
import torch.nn as nn
from glob import glob
from os.path import expanduser, join, basename, dirname
import xarray as xr
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
from itertools import product
from torch.utils.data import TensorDataset, DataLoader
from astra.torch.models import EfficientNetClassifier,EfficientNet_B0_Weights
from astra.torch.utils import train_fn

import torchvision.models as models
from astra.torch.metrics import accuracy_score, f1_score, precision_score, recall_score,classification_report

In [3]:
def get_common_label_files(path1, path2):
    files1 = glob(join(path1, "*.nc"))
    files2 = glob(join(path2, "*.nc"))
    
    f1_base_files = [basename(f) for f in files1]
    f2_base_files = [basename(f) for f in files2]
    
    common_files = set(f1_base_files).intersection(f2_base_files)
    common_label_files = []
    for file in common_files:
        ds1 = xr.open_dataset(join(path1, file))
        ds2 = xr.open_dataset(join(path2, file))
        if np.all(ds1.label.values == ds2.label.values):
            common_label_files.append(file)
    
    return list(map(lambda f: join(path1, f), common_label_files))

def get_disjoint_files(path1, path2):
    files1 = glob(join(path1, "*.nc"))
    files2 = glob(join(path2, "*.nc"))
    
    f1_base_files = [basename(f) for f in files1]
    f2_base_files = [basename(f) for f in files2]
    
    disjoint_files = set(f1_base_files).symmetric_difference(f2_base_files)
    
    f1_disjoint = [f for f in disjoint_files if f in f1_base_files]
    f1_disjoint = list(map(lambda f: join(path1, f), f1_disjoint))

    f2_disjoint = [f for f in disjoint_files if f in f2_base_files]
    f2_disjoint = list(map(lambda f: join(path2, f), f2_disjoint))
    
    return f1_disjoint + f2_disjoint

In [4]:
base_path = expanduser("/home/rishabh.mondal/bangladesh_labels/bkdb/india_labels/region/power_plant")
paths = {"suraj": ("shataxi", "rishabh")}

all_labeled_files = []
for moderator, annotators in paths.items():
    # Get moderator files
    moderator_path = join(base_path, "moderated", moderator)
    moderator_files = glob(join(moderator_path, "*.nc"))
    
    # Get annotator common label files
    annotator1_path = join(base_path, annotators[0])
    annotator2_path = join(base_path, annotators[1])
    
    common_base_files = get_common_label_files(annotator1_path, annotator2_path)
    
    # Get disjoint files
    disjoint_files = get_disjoint_files(annotator1_path, annotator2_path)
    
    all_files = moderator_files + common_base_files
    if use_disjoint_files:
        all_files.extend(disjoint_files)
    assert len(all_files) == len(set(all_files))
    all_labeled_files.extend(all_files)
    
    print("Moderator", moderator)
    print(" "*5, "Moderator files", len(moderator_files))
    print(" "*5, "Common label files", len(common_base_files))
    print(" "*5, "Disjoint files", len(disjoint_files))
    print(" "*5, f"Total files from {moderator} and {annotators}", len(all_files))
    print(" "*5, "Total annotatated files", len(all_labeled_files))
    
print("Total dataset size", len(all_labeled_files) * 25)

Moderator suraj
      Moderator files 0
      Common label files 379
      Disjoint files 1
      Total files from suraj and ('shataxi', 'rishabh') 379
      Total annotatated files 379
Total dataset size 9475


In [5]:
images_path = expanduser("~/bkdb/india_power_plant_data")
# load_path = "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/temporary"
files = all_labeled_files
# print(files)
print(len(files))

379


In [6]:
def get_index_and_image(file):
    index = []
    images = []
    labels = []
    base_name = basename(file)
    # print(base_name)
    image_path = join(images_path, base_name).replace(".nc", ".zarr")
    # print(image_path)
    label_ds = xr.open_dataset(file)
    # print (label_ds)
    image_ds = xr.open_zarr(image_path, consolidated=False)
    # image = image_ds.data.reshape(-1, 224, 224, 3)
    for lat_lag, lon_lag in product(range(-2, 3), repeat=2):
        index.append(base_name.replace(".nc", "")+f"_{lat_lag}_{lon_lag}")
        images.append(torch.tensor(image_ds.sel(lat_lag=lat_lag, lon_lag=lon_lag)['data'].values)[np.newaxis, ...])
        labels.append(torch.tensor((label_ds.sel(lat_lag=lat_lag, lon_lag=lon_lag)['label'].values != "O").astype(np.uint8)))
        
    return index, images, labels



def get_data():
    out = Parallel(n_jobs=32)(delayed(get_index_and_image)(file) for file in tqdm(files, total=len(files)))
    index = np.concatenate([np.array(idx) for idx, _, _ in out])
    images = torch.concat([torch.einsum("nhwc->nchw", torch.concat(imgs)) for _, imgs, _ in out])
    # scale
    # images = images / 255
    # mean normalize
    # images = (images - images.mean(dim=(0, 2, 3), keepdim=True)) / images.std(dim=(0, 2, 3), keepdim=True)
    
    labels = np.concatenate([np.array(lbl) for _, _, lbl in out])
    labels = torch.tensor(labels, dtype=torch.uint8)
    #check the all dytpes
    print(index.dtype, images.dtype, labels.dtype)
    return index, images, labels

index, images, labels = get_data()
print(images.dtype)
print(index.shape, images.shape, labels.shape)    

100%|██████████| 379/379 [00:07<00:00, 54.14it/s]


<U17 torch.uint8 torch.uint8
torch.uint8
(9475,) torch.Size([9475, 3, 224, 224]) torch.Size([9475])


In [8]:
# save the tensors data 
print(images.dtype)
save_path="/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data/pp.pt"
torch.save({
    'index': index,
    'images': images,
    'labels': labels
}, save_path)

torch.float32


In [40]:

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

image_path = "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/predicted_positive/baghpat/positive_image_5488.jpg"

# Read the image
image = mpimg.imread(image_path)

# Plot the image
plt.figure(dpi=300)
plt.imshow(image)
plt.axis('off')

# Save the image as PDF with resolution
output_path = "/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/positive_image_5488.pdf"
plt.savefig(output_path, dpi=300, format='pdf', bbox_inches='tight', pad_inches=0)
plt.close()


In [38]:
# # scale
# images = images / 255
# # mean normalize
# images = (images - images.mean(dim=(0, 2, 3), keepdim=True)) / images.std(dim=(0, 2, 3), keepdim=True)

In [63]:
# train_images = images[:int(0.2*len(images)):]
# test_images = images[int(0.2*len(images)):]
# train_images.shape, test_images.shape
# train_labels = labels[:int(0.2*len(labels))]
# test_labels = labels[int(0.2*len(labels)):]
# train_labels.shape, test_labels.shape

In [64]:
test_data = TensorDataset(images, labels)
test_loader = DataLoader(test_data, batch_size=512, shuffle=False, num_workers=32)


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
import torchvision.models as models

efficientnet = models.efficientnet_b0(pretrained=True)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

efficientnet.classifier[1]=nn.Linear(in_features=1280, out_features=2, bias=True)

/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(


In [12]:
model=efficientnet.to('cuda')

In [5]:
train_model = EfficientNetClassifier(
    models.efficientnet_b0,EfficientNet_B0_Weights, n_classes=2, activation=nn.ReLU(), dropout=0.1
).to(device)

In [13]:
model.load_state_dict(torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/Saved_model/bangladesh_whole_model_no_pretrain.pth"))

RuntimeError: Error(s) in loading state_dict for EfficientNet:
	Missing key(s) in state_dict: "classifier.1.weight", "classifier.1.bias". 
	Unexpected key(s) in state_dict: "linear.0.weight", "linear.0.bias", "classifier.0.weight", "classifier.0.bias". 

: 

In [33]:
# Load the saved tensors
loaded_data1 = torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data/test_data.pt")

# Access the tensors
index1 = loaded_data1['index']
images1 = loaded_data1['images']
labels1 = loaded_data1['labels']
images1 = images1 / 255
# mean normalize
images1 = (images1 - images1.mean(dim=(0, 2, 3), keepdim=True)) / images1.std(dim=(0, 2, 3), keepdim=True)
#print shape of tensors
print(images1.shape)
print(labels1.shape)
#print the labels
print(labels1)
#count the number of unique labels
# count the number of 1s and 0s
# unique, counts = np.unique(labels, return_counts=True)
# print(dict(zip(unique, counts)))

# subset_labels = labels
# num_ones = np.count_nonzero(subset_labels == 1)
# num_zeros = np.count_nonzero(subset_labels == 0)

# print(f"Number of 1s(train ): {num_ones}")
# print(f"Number of 0s(train): {num_zeros}")

torch.Size([10025, 3, 224, 224])
torch.Size([10025])
tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.uint8)


In [8]:
test_data1 = TensorDataset(images1, labels1)
test_loader1 = DataLoader(test_data1, batch_size=512, shuffle=False, num_workers=32)

In [9]:
with torch.no_grad():
    pred_classes =train_model.predict_class(
        dataloader=test_loader1, batch_size=254, verbose=True
    ).to(device)

test_labels = labels1.to(device)
print("Accuracy: ", accuracy_score(pred_classes,test_labels))
print("Precision: ", precision_score(pred_classes,test_labels))
print("Recall: ", recall_score(pred_classes,test_labels))
print("F1: ", f1_score(pred_classes,test_labels))
print("classification report: ", classification_report(pred_classes,test_labels))

100%|██████████| 20/20 [00:10<00:00,  1.83it/s]


Accuracy:  tensor(0.9414, device='cuda:0')
Precision:  tensor(0.8970, device='cuda:0')
Recall:  tensor(0.4933, device='cuda:0')
F1:  tensor(0.6365, device='cuda:0')
classification report:  {'accuracy': tensor(0.9414, device='cuda:0'), 'precision': tensor(0.8970, device='cuda:0'), 'recall': tensor(0.4933, device='cuda:0'), 'f1': tensor(0.6365, device='cuda:0')}


In [69]:
model.load_state_dict(torch.load("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/Saved_model/strafied_efficient_whole_bangladesh.pth"))

<All keys matched successfully>

In [70]:
with torch.no_grad():
    model.eval()
    true_labels = []
    pred_labels = []
    for i, (inputs, labels) in enumerate(test_loader):
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        output = model(inputs)
        _, preds = torch.max(output, 1)
        true_labels.extend(labels.data.cpu().numpy())
        pred_labels.extend(preds.data.cpu().numpy())
test_accuracy = accuracy_score(true_labels, pred_labels)
# test_accuracies.append(test_accuracy)
test_precision = precision_score(true_labels, pred_labels)
# test_precisions.append(test_precision)
test_recall = recall_score(true_labels, pred_labels)
# test_recalls.append(test_recall)
test_f1 = f1_score(true_labels, pred_labels)
# test_f1_scores.append(test_f1)


/home/rishabh.mondal/miniconda3/envs/torch_space/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [71]:
total_predicted_positive = sum(pred_labels)
total_predicted_negative = len(pred_labels) - total_predicted_positive

print("Total Predicted Positive:", total_predicted_positive)
print("Total Predicted Negative:", total_predicted_negative)



Total Predicted Positive: 14
Total Predicted Negative: 9461


In [72]:
classification_report(true_labels, pred_labels)

AttributeError: 'bool' object has no attribute 'float'

: 

In [44]:
# with torch.no_grad():
#     pred_classes =model.predict_class(
#         dataloader=test_loader, batch_size=512, verbose=True
#     ).to(device)

AttributeError: 'EfficientNet' object has no attribute 'predict_class'

In [47]:
zero_predictions = torch.sum(pred_labels == 0)
one_predictions = torch.sum(pred_labels == 1)

print("Number of zero predictions:", zero_predictions.item())
print("Number of one predictions:", one_predictions.item())

TypeError: sum(): argument 'input' (position 1) must be Tensor, not bool

In [31]:
path='/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/tensor_data_final/test_data.pt'
loaded_data1 = torch.load(path)
images1 = loaded_data1['images']
labels1 = loaded_data1['labels']
positive_images = images1[labels1 == 1]


In [32]:
# Filter the images based on the label
# pred_classes = pred_classes.cpu().numpy()
label_1_images = images[labels1 == 1]
print(label_1_images.shape)
# Plot the filtered images
fig, axes = plt.subplots(5, 5, figsize=(25,25))

for i in range(5):
    for j in range(5):
        # Adjust the index calculation to match the desired grid
        index = i * 4 + j
        if index < len(label_1_images):
            # Transpose the dimensions to (32, 32, 3) for RGB image
            image_to_display = label_1_images[index].permute(1, 2, 0)
            image_to_display = (image_to_display - image_to_display.min()) / (image_to_display.max() - image_to_display.min())

            axes[i, j].imshow(image_to_display)
            #put the label
            # axes[i, j].set_title(label_1_images[index].item())
            axes[i, j].axis('off')
        else:
            # If there are fewer than 20 images, remove empty subplots
            fig.delaxes(axes[i, j])
plt.tight_layout()
plt.show()

IndexError: The shape of the mask [4500] at index 0 does not match the shape of the indexed tensor [9475, 3, 224, 224] at index 0